## Bilinear Interpolation

In [1]:
x1=14 
x2=15 
y1=21
y2=20 

f_Q12 = 91
f_Q22 = 210
f_Q11 = 162
f_Q21 = 95

# giá trị cần tìm cho tọa độ x và y
x = 14.5
y = 20.2 

f_R1 = f_Q11*(x2-x)/(x2-x1) + f_Q21*(x-x1)/(x2-x1)
f_R2 = f_Q12*(x2-x)/(x2-x1) + f_Q22*(x-x1)/(x2-x1)
f_P = f_R1*(y2-y)/(y2-y1) + f_R2*(y-y1)/(y2-y1)

print(f"f_R1={f_R1}")
print(f"f_R2={f_R2}")
print(f"f_P={f_P}")

f_R1=128.5
f_R2=150.5
f_P=146.10000000000002


In [ ]:
def bilinear_interpolation(coors, P, f):
    x1, y1, x2, y2 = coors    
    x, y = P
    
    f_Q12 = f[y2, x1]
    f_Q22 = f[y2, x2]
    f_Q11 = f[y1, x1]
    f_Q21 = f[y1, x2]

    f_R1 = f_Q11*(x2-x)/(x2-x1) + f_Q21*(x-x1)/(x2-x1)
    f_R2 = f_Q12*(x2-x)/(x2-x1) + f_Q22*(x-x1)/(x2-x1)
    f_P = f_R1*(y2-y)/(y2-y1) + f_R2*(y-y1)/(y2-y1)
   
    return f_P


def bilinear_interpolation_resize(image, new_h, new_w):
    old_h, old_w = image.shape[:2]
    new_image = np.zeros((new_h, new_w))
    
    #Calculate horizontal and vertical scaling factor
    w_scale_factor = old_w/new_w 
    h_scale_factor = old_h/new_h

    for y_new in range(new_h):
        for x_new in range(new_w):
    
            #map the coordinates back to the original image
            y_new_scaled = y_new * h_scale_factor
            x_new_scaled = x_new * w_scale_factor
    
            #calculate the coordinate values for 4 surrounding pixels.
            x1 = math.floor(x_new_scaled)
            x2 = min(old_w - 1, math.ceil(x_new_scaled))
            y2 = math.floor(y_new_scaled)
            y1 = min(old_h - 1, math.ceil(y_new_scaled))
            
            if (x2 == x1) and (y1 == y2):
                f_P = image[int(y_new_scaled), int(x_new_scaled)]
           
            elif (x2 == x1):  # (vertical) linear interpolation 
                f_R1 = image[int(y1), int(x_new_scaled)]
                f_R2 = image[int(y2), int(x_new_scaled)]
                f_P = f_R1*(y2-y_new_scaled)/(y2-y1) + f_R2*(y_new_scaled-y1)/(y2-y1)
                
            
            elif (y1 == y2): # (horizontal) linear interpolation 
                f_RX1 = image[int(y_new_scaled), int(x1)]
                f_RX2 = image[int(y_new_scaled), int(x2)]
                f_P = f_RX1*(x2-x_new_scaled)/(x2-x1) + f_RX2*(x_new_scaled-x1)/(x2-x1)
                
            else:
                coors = [x1, y1, x2, y2]
                P = [x_new_scaled, y_new_scaled]
                f_P = bilinear_interpolation(coors, P, image)
                
            new_image[y_new,x_new] = f_P
    return new_image.astype(np.uint8)